<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"
alt="Rubin Observatory logo, a graphical representation of turning stars into data."> <br><br>
**Description:** Generate parameters for the _Rubin Observatory DP1_ paper. <br>
**Authors:** Leanne Guy <br>
**Last verified to run:** 2025-05-14 <br>
**LSST Science Pipelines version:** v29.0.0<br>
**Container size:** medium <br>

# DP1 parameters

## Setup

In [ ]:
! pip install inflect > /dev/null 2>&1

In [ ]:
# Generic python packages
import os
import csv
import requests
import yaml
import numpy as np
import pandas as pd
import numpy as np
from pathlib import Path


# LSST Science Pipelines 
from lsst.daf.butler import Butler

# RSP
from lsst.rsp import get_tap_service

import warnings
warnings.filterwarnings("ignore")

### Define utility functions

In [ ]:
# Custom formatter: no trailing zeros if not needed
def custom_float(x):
    if isinstance(x, float):
        s = f"{x:.3f}" #.rstrip('0').rstrip('.')
        return s
    return x

In [ ]:
# Function to round to N significant figures
def round_sf(x, sig=3):
    return np.round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

In [ ]:
# Template for a parameter
def format_param(key, value):
    return f"\\newcommand{{\\{key}}}{{{value}\\xspace}}"

### Set up DP1 Butler

In [ ]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

In [ ]:
## Consdb
cdb = get_tap_service("consdbtap")
assert cdb is not None

In [ ]:
# Look at the cdb_lsstcomcam schema
query = '''SELECT * FROM tap_schema.tables 
        WHERE tap_schema.tables.schema_name = 'cdb_lsstcomcam' 
        AND table_name not like '*_flexdata'
        ORDER BY table_index ASC
        '''
tables = cdb.search(query).to_table()
tables

# Set up paramters dictionary

In [ ]:
# params dictionary
params = {}

# Compute parameters

In [ ]:
# Date range for campaign
list_butler = []
query = '''SELECT * FROM cdb_lsstcomcam.exposure
where esposure_id in list_butler
ORDER BY day_obs DESC
        '''
exposures = cdb.search(query).to_table().to_pandas()
exposures.columns
#exposures['day_obs'].unique()

In [ ]:
# Extract filter and pointing information about the fileds from the DP1 exposures in the Butler
exposures = registry.queryDimensionRecords('exposure')
exp_df = pd.DataFrame(columns=['id', 'target', 'physical_filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        exp_df.loc[count] = [info.id, info.target_name, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec,]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])

# slew_icrs is the ECDFS field -- combine them
exp_df.loc[exp_df['target'] == 'slew_icrs', 'target'] = 'ECDFS'
dp1_fields = exp_df.target.unique()

params["nfields"] = len(dp1_fields)

# Write the paramters to file in latex format

In [ ]:
parameters_filepath = str(Path('../parameters.tex').resolve())
with open(parameters_filepath, "w") as file:
    file.write("%%%%% This table is auto generated from data, DO NOT EDIT" + "\n")
    for key, value in params.items():
        formatted_line = format_param(key, value)
        file.write(formatted_line + "\n")